In [5]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [43]:
ref = open("./pedestrian/temporalROI.txt").read().split(" ")
start, end = int(ref[0]), int(ref[1])

tp = 0
tn = 0 
fn = 0
fp = 0

last = cv2.imread(f"./pedestrian/input/in000299.jpg", cv2.IMREAD_GRAYSCALE)
for i in range(start, end):
    file_number = str(i).zfill(6)
    image = cv2.imread(f"./pedestrian/input/in{file_number}.jpg", cv2.IMREAD_GRAYSCALE)
    
    image = image.astype("int")
    last = last.astype("int")
    
    diff = np.abs(image - last)
    diff = diff.astype("uint8")
    
    diff = cv2.medianBlur(diff, 5)

    thr, bin_img = cv2.threshold(diff, 15, 255, cv2.THRESH_BINARY)
    
    res_img = bin_img
    
    res_img = cv2.medianBlur(res_img, 5)
    
    res_img = cv2.dilate(res_img, np.ones((3,3)), iterations=3)
    
    retval, labels, stats, centroids = cv2.connectedComponentsWithStats (res_img)

    
    
    res_copy = labels
    
    if(stats.shape[0]>1):
        tab = stats[1:,4]
        pi = np.argmax(tab)
        pi = pi + 1
        
        cv2.rectangle(res_copy,(stats[pi,0], stats[pi,1]), (stats[pi,0]+stats[pi,2], stats[pi,1]+stats[pi,3]),(255,0,0),2)
        cv2.putText(res_copy, f"{stats[pi,4]}", (stats[pi,0], stats[pi,1]), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0))
        
        cv2.putText(res_copy, f"{pi}", (int(centroids[pi, 0]), int(centroids[pi,1])), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0))
    
    actual = cv2.imread(f"./pedestrian/groundtruth/gt{file_number}.png", cv2.IMREAD_GRAYSCALE)
    
    true_positive_sum = np.sum(np.logical_and((res_img>0), (actual>0)))
    
    true_negative_sum = np.sum(np.logical_and((res_img==0), (actual==0)))
    
    false_positive_sum = np.sum(np.logical_and((res_img>0), (actual==0)))
    
    false_negative_sum = np.sum(np.logical_and((res_img==0), (actual>0)))

    tp += true_positive_sum
    tn += true_negative_sum
    fp += false_positive_sum
    fn += false_negative_sum
    
    
    cv2.imshow("image", res_img)
    
#     cv2.imshow("image", np.uint8(labels/retval*255))
    
    last = image
    cv2.waitKey(1)

cv2.destroyAllWindows()

In [45]:
p = tp/(tp+fp)
r = tp/(tp+fn)
f1 = (2*p*r)/(p+r)
f1

0.7204031809342106

In [17]:
start
end

'1099'